In [62]:
#adapted from @bact at https://colab.research.google.com/drive/1hdtmwTXHLrqNmDhDqHnTQGpDVy1aJc4t
import json
import pandas as pd
import numpy as np
import re
import pycrfsuite
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
with open('data/talks.transcript.th-en.24-10-2019_11:12.json', 'r') as f:
    data = json.load(f)

In [63]:
talk_idx = []
idx = []
th_phrases = []
en_phrases = []

for i in range(len(data)):
    nb_phrases = min(len(data[i]['en']),len(data[i]['th']))
    for j in range(nb_phrases):
        talk_idx.append(i)
        th_phrases.append(data[i]['th'][j])
        en_phrases.append(data[i]['en'][j])
        if data[i]['en'][j][-1]=='.':
            idx.append(1)
        else:
            idx.append(0)

In [64]:
phrase_df = pd.DataFrame({'talk_idx':talk_idx,'en_phrase':en_phrases ,'th_phrase':th_phrases, 'idx':idx})
phrase_df.idx.sum()

136463

In [65]:
all_sentences = []
for i in range(phrase_df.talk_idx.max()):
    df = phrase_df[phrase_df.talk_idx==i]
    sentences = []
    for j,row in df.iterrows():
        sentences.append(row.th_phrase)
        if row.idx==1:
            sentences.append(' |')
    joined_sentences = ''.join(sentences)
    #remove parantheses like (audience claps)
    joined_sentences = re.sub(r'\([^)]*\)', '', joined_sentences)
    #skip if talk is nothing but parantheses
    if joined_sentences=='': continue
    #remove | at the last sentence if present
    if joined_sentences[-1]=='|': joined_sentences = joined_sentences[:-1]
    all_sentences.append(joined_sentences)

In [7]:
len(all_sentences)

1544

In [66]:
all_sentences[0]

'บนหลังอาชาแห้งกร่องดอนกิโฆเต้ พระเอกของเราบุกตะลุยสู้กับกองทัพยักษ์ |ในสายตาของเขา มันเป็นหน้าที่ของเขาที่จะปราบอสูรร้ายเหล่านี้ในนามแห่งหญิงอันเป็นที่รักของเขา ดุลสิเนอา |ทว่า การกระทำอันหาญกล้านี้ก็สูญเปล่า |เมื่อซานโซ่ ปันซ่า ผู้รับใช้ของเขาอธิบายครั้งแล้วครั้งเล่า ว่าสิ่งเหล่านี้จะเป็นยักษ์ก็หาไม่พวกมันเป็นเพียงกังหันลมเท่านั้น |ดอนกิโฆเต้ หาได้เสียความแน่วแน่แทงทวนของเขาเข้าไปยังใบพัดอย่างจัง |ด้วยพลังใจที่ไม่เคยถดถอยอัศวินผู้นั้นยืนขึ้นอย่างภาคภูมิและยิ่งเชื่อมั่นในปฏิบัติการของเขามากขึ้น |ลำดับเหตุการณ์นี้ครอบคลุมเรื่องราวส่วนใหญ่ของดอนกิโฆเต้ ที่เป็นที่รักมหากาพย์ ไร้ตรรกะ และมีชีวิตชีวาของ อลองโซ กีฆานาผู้กลายเป็น ดอนกิโฆเต้ แห่งลามันช่าผู้ซุ่มซ่ามแต่กล้าหาญหรือที่รู้จักกันในนาม ขุนนางต่ำศักดิ์นักฝัน |แต่เดิมวรรณกรรมนี้มีสองเล่มบรรยายเรื่องราวของดอนกิโฆเต้ในขณะที่เขาเดินทางผ่านตอนกลาง และตอนเหนือของสเปนเพื่อต่อสู้กับบรรดาปิศาจร้าย |แม้จินตนาการในเรื่อง ดอนกิโฆเต้ จะสูงล้ำเหนือเมฆผู้ประพันธ์ มิเกล เด เซร์บันเตสก็ไม่เคยนึกฝันว่าหนังสือของเขาจะกลายเป็นนิยายที่ขายดีที่สุดตลอดกาล 

In [67]:
all_tuples = []
for i in range(len(all_sentences)):
    tuples = []
    for s in all_sentences[i].split('|'):
        s_lst = word_tokenize(s)
        for j in range(len(s_lst)):
            lab = 'B' if j==0 else 'I'
            tuples.append((s_lst[j],lab))
    all_tuples.append(tuples)

In [68]:
len(all_tuples)

1544

In [69]:
# Extract features of each wordacter from text, in CRFSuite format
def extract_features(doc):
    doc_pos = [p for (w,p) in pos_tag(doc)]
    doc_features = []
    for i, word in enumerate(doc):
        word_features = [
            'bias',
            'word=' + word,
            'pos=' + doc_pos[i],
        ]
        if i > 0:
            feats = [
                'word[-1]=' + doc[i-1],
                'pos[-1]=' + doc_pos[i-1],
            ]
            word_features.extend(feats)

        if i > 1:
            feats = [
                'word[-2]=' + doc[i-2],
                'pos[-2]=' + doc_pos[i-2],
            ]
            word_features.extend(feats)

        if i < len(doc)-1:
            feats = [
                'word[+1]=' + doc[i+1],
                'pos[+1]=' + doc_pos[i+1],
            ]
            word_features.extend(feats)
            
        if i < len(doc)-2:
            feats = [
                'word[+2]=' + doc[i+2],  
                'pos[+2]=' + doc_pos[i+2],
            ]
            word_features.extend(feats)
        doc_features.append(word_features)
    return doc_features

In [70]:
#target
y = [[l for (w,l) in t] for t in all_tuples]
#features
x_pre = [[w for (w,l) in t] for t in all_tuples]
x = [extract_features(x_) for x_ in x_pre]

In [71]:
len(x),len(y)

(1544, 1544)

In [117]:
# Split train and test set at 80/20 proportion
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1412)

In [165]:
# Train model
trainer = pycrfsuite.Trainer(verbose=True)
#trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(x_train, y_train):
  trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 1,
    'c2': 0,
    'max_iterations': 200,
    'feature.possible_transitions': True,
})

trainer.train('sentenceseg-crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 187577
Seconds required: 7.105

L-BFGS optimization
c1: 1.000000
c2: 0.000000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 458444.418198
Feature norm: 1.000000
Error norm: 231626.694344
Active features: 112956
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 2.029

***** Iteration #2 *****
Loss: 443304.813521
Feature norm: 1.054244
Error norm: 204120.359938
Active features: 97510
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.061

***** Iteration #3 *****
Loss: 366662.986677
Feature norm: 1.389790
Error norm: 199296.086612
Active features: 54832
Line search trials: 1
Line search step: 1.000000
Seconds requ

***** Iteration #41 *****
Loss: 141969.240497
Feature norm: 39.097041
Error norm: 12364.733006
Active features: 28169
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.251

***** Iteration #42 *****
Loss: 141115.443226
Feature norm: 41.741811
Error norm: 4062.308057
Active features: 28071
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.293

***** Iteration #43 *****
Loss: 140275.743363
Feature norm: 45.164504
Error norm: 6463.325755
Active features: 28021
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.030

***** Iteration #44 *****
Loss: 139477.332102
Feature norm: 49.234803
Error norm: 7382.128069
Active features: 27864
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.041

***** Iteration #45 *****
Loss: 138812.522301
Feature norm: 52.551229
Error norm: 5597.666164
Active features: 27736
Line search trials: 1
Line search step: 1.

***** Iteration #82 *****
Loss: 134090.641841
Feature norm: 85.732620
Error norm: 2881.225931
Active features: 24286
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.953

***** Iteration #83 *****
Loss: 134067.115328
Feature norm: 85.794006
Error norm: 4501.094206
Active features: 24308
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.069

***** Iteration #84 *****
Loss: 134024.553611
Feature norm: 85.870343
Error norm: 3821.029255
Active features: 24271
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.132

***** Iteration #85 *****
Loss: 133995.678326
Feature norm: 85.959078
Error norm: 5450.230357
Active features: 24255
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.034

***** Iteration #86 *****
Loss: 133954.246748
Feature norm: 86.066662
Error norm: 4872.373651
Active features: 24243
Line search trials: 2
Line search step: 0.5

***** Iteration #122 *****
Loss: 132908.051262
Feature norm: 88.189738
Error norm: 3869.354755
Active features: 23938
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 3.417

***** Iteration #123 *****
Loss: 132883.915713
Feature norm: 88.212410
Error norm: 4478.826346
Active features: 23941
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.741

***** Iteration #124 *****
Loss: 132856.322026
Feature norm: 88.244259
Error norm: 3712.681478
Active features: 23929
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.832

***** Iteration #125 *****
Loss: 132832.111461
Feature norm: 88.269436
Error norm: 4163.042587
Active features: 23925
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.860

***** Iteration #126 *****
Loss: 132807.066142
Feature norm: 88.304245
Error norm: 3531.756239
Active features: 23911
Line search trials: 2
Line search step

***** Iteration #163 *****
Loss: 132074.143041
Feature norm: 89.760269
Error norm: 3027.070542
Active features: 23597
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.404

***** Iteration #164 *****
Loss: 132060.732848
Feature norm: 89.782474
Error norm: 3081.904207
Active features: 23594
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.299

***** Iteration #165 *****
Loss: 132046.638321
Feature norm: 89.775153
Error norm: 3004.049349
Active features: 23591
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.232

***** Iteration #166 *****
Loss: 132033.940531
Feature norm: 89.795124
Error norm: 3130.399477
Active features: 23587
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.317

***** Iteration #167 *****
Loss: 132020.172919
Feature norm: 89.790831
Error norm: 2995.205138
Active features: 23593
Line search trials: 2
Line search step

In [166]:
# Predict (using test set)
tagger = pycrfsuite.Tagger()
tagger.open('sentenceseg-crf.model')
y_pred = [tagger.tag(xseq) for xseq in x_test]

In [167]:
# Evaluate
labels = {'B': 0, "I": 1} # classification_report() needs values in 0s and 1s
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

print(classification_report(
    truths, predictions,
    target_names=["B", "I"]))

              precision    recall  f1-score   support

           B       0.71      0.67      0.69     27763
           I       0.99      0.99      0.99    649564

    accuracy                           0.98    677327
   macro avg       0.85      0.83      0.84    677327
weighted avg       0.97      0.98      0.97    677327



In [168]:
def sentence_tokenize(s):
    toks = word_tokenize(s)
    feat = extract_features(toks)
    labs = tagger.tag(feat)
    sentences = []
    sentence = ''
    for i, w in enumerate(toks):
        if labs[i] == 'B':
            if sentence:
                sentences.append(sentence)
            sentence = ''
        sentence = sentence + w
    if sentence:
        sentences.append(sentence)
    return sentences

In [169]:
s = 'เธอคือหุ่นยนต์รูปแบบใหม่ที่ฉันคนนี้สร้างขึ้นมาเธอมีความสามารถในการคิดและรู้สึกเหมือนมนุษย์ เธอสามารถตัดสินใจด้วยตัวเอง แต่ถึงอย่างนั้นมันก็เป็นดาบสองคม ที่อาจเป็นอันตรายถ้าเธอคิดว่ามนุษย์เป็นภัย จนแหกกฎข้อแรกที่หุ่นยนต์ห้ามทำร้ายมนุษย์ ฉันจำเป็นต้องใช้เวลาทดสอบระบบอีก 30 ปีเพื่อความมั่นใจว่าเธอจะไม่เป็นอันตรายกับมนุษย์ แต่ฉันคงจะมีชีวิตอยู่ไม่ถึงวันนั้น ดังนั้นฉันจึงปิดผนึกเธอเอาไว้ในแคปซูลจนกว่าระบบต่าง ๆ จะพร้อมเสียก่อน'

In [170]:
sentence_tokenize(s)

['เธอคือหุ่นยนต์รูปแบบใหม่ที่ฉันคนนี้สร้างขึ้นมาเธอมีความสามารถในการคิดและรู้สึกเหมือนมนุษย์<space>',
 'เธอสามารถตัดสินใจด้วยตัวเอง<space>',
 'แต่ถึงอย่างนั้นมันก็เป็นดาบสองคม<space>ที่อาจเป็นอันตรายถ้าเธอคิดว่ามนุษย์เป็นภัย<space>จนแหกกฎข้อแรกที่หุ่นยนต์ห้ามทำร้ายมนุษย์<space>',
 'ฉันจำเป็นต้องใช้เวลาทดสอบระบบอีก<space>30<space>ปีเพื่อความมั่นใจว่าเธอจะไม่เป็นอันตรายกับมนุษย์<space>',
 'แต่ฉันคงจะมีชีวิตอยู่ไม่ถึงวันนั้น<space>',
 'ดังนั้นฉันจึงปิดผนึกเธอเอาไว้ในแคปซูลจนกว่าระบบต่าง ๆ<space>จะพร้อมเสียก่อน']